<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Get-web-page-and-save-the-response-to-local" data-toc-modified-id="Get-web-page-and-save-the-response-to-local-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Get web page and save the response to local</a></span></li><li><span><a href="#Read-webpage-from-local" data-toc-modified-id="Read-webpage-from-local-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Read webpage from local</a></span></li><li><span><a href="#Extract-Data" data-toc-modified-id="Extract-Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Extract Data</a></span></li></ul></div>

In [128]:
import json

import requests
from bs4 import BeautifulSoup
import urllib

import pandas as pd
from pandas.io.json import json_normalize

#### Get web page and save the response to local

In [115]:
"""
Webpage URLs
"""
url_alza_laptop_home_and_office = 'https://www.alza.co.uk/basic-home-and-office-laptops/18843464.htm'
url_alza_laptop_gaming = 'https://www.alza.co.uk/gaming-laptops/18848814.htm'
url_alza_laptop_professional = 'https://www.alza.co.uk/professional-laptops/18853299.htm'

"""
Local paths to store webpages
"""
local_path_alza_laptop_home_and_office = './basic-home-and-office-laptops.html'
local_path_alza_laptop_gaming = './gaming-laptops.html'
local_path_alza_laptop_professional = './professional-laptops.html'

In [114]:
def scrape2local(url, path_des):
    file_obj = requests.get(url)
    with open(path_des, 'w', encoding= 'utf-8') as f:
        f.write(file_obj.content.decode('utf-8'))

In [119]:
# ONLY NEED TO EXECUTE ONCE
# scrape2local(url_alza_laptop_home_and_office, local_path_alza_laptop_home_and_office)
# scrape2local(url_alza_laptop_gaming, local_path_alza_laptop_gaming)
# scrape2local(url_alza_laptop_professional, local_path_alza_laptop_professional)

#### Read webpage from local

In [123]:
with open(local_path_alza_laptop_home_and_office, 'r', encoding='utf-8') as file_HO:
    html_HO = file_HO.read()

In [124]:
with open(local_path_alza_laptop_gaming, 'r', encoding='utf-8') as file_G:
    html_G = file_G.read()

In [125]:
with open(local_path_alza_laptop_professional, 'r', encoding='utf-8') as file_P:
    html_P = file_P.read()

In [126]:
soup_HO = BeautifulSoup(html_HO, 'lxml')
soup_G = BeautifulSoup(html_G, 'lxml')
soup_P = BeautifulSoup(html_P, 'lxml')

#### Extract Data

In [174]:
cols = ['name', 'category', 'rank']
df_laptop = pd.DataFrame(columns= cols)
df_laptop

,name,category,rank


In [175]:
def extract_top3(soup, category, cols):
    item_results = soup.find('div', id = 'boxes')
    item_results_top = item_results.find_all('div', class_='top')
    rank_count = 1
    res = []
    for item in item_results_top[:3]:
        laptop_name = item.find('a', class_= 'name').get_text()
        res.append({cols[0] : laptop_name, 
                    cols[1] : category, 
                   cols[2] : rank_count})
        rank_count += 1
    return pd.DataFrame(data= res, columns= cols)
        
df_laptop = df_laptop.append(extract_top3(soup_HO, 'home_and_office', cols), 
                             ignore_index= True)
df_laptop = df_laptop.append(extract_top3(soup_G, 'gaming', cols),
                            ignore_index= True)
df_laptop = df_laptop.append(extract_top3(soup_P, 'professional', cols),
                            ignore_index= True)

In [176]:
df_laptop

,name,category,rank
0,Microsoft Surface Go 128GB 8GB LTE,home_and_office,1
1,Dell Latitude 5590,home_and_office,2
2,Dell G7 17 (7790) Gaming Black,home_and_office,3
3,Lenovo IdeaPad 720s-15IKB Silver Metallic,gaming,1
4,Lenovo Legion Y530-15ICH Black,gaming,2
5,Dell G7 17 (7790) Gaming Black,gaming,3
6,Lenovo V330-14IKB Iron Grey,professional,1
7,Dell Latitude 5590,professional,2
8,Lenovo ThinkPad T480,professional,3
